# Develop the final state class
## There are 3 parts of the class the user is primarily responsible for
1. Calculations specific to each topology e.g. start time, missing mass,....
2. Calculations common to all topologies, typically the Kinematics of the reaction
3. The Data to be saved in an output tree

## Topology Specific
This requires defining the Topo_X functions where C is the number of the topology.
These are saved in a seperate file ToposXXX.h. We will edit this directly in this Jupyter notebook.

Here you can set the reaction start time to allow calculation of DeltaTime for PID purposes. It may be you want to take this from one particular type of particle such as scattered electron or tagger photon.
You should also calculate the missing mass for this topology and other masses specific to a topology.

Again you will need to change XXX to your class name

In [ ]:
//gROOT->ProcessLine(".x $HSCODE/hsfinalstate/LoadFinalState.C+");

%edit ToposK2.h

In [ ]:
%%file ToposK2.h
#include "K2.h"
void K2::Topo_0(){
  //For topology Beam:Proton:Kp:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKp,&fKm);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKp.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKp.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
}
void K2::Topo_1(){
  //For topology Beam:Proton:Kp

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKp);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKp.TakeCorrectedP();//Apply eloss
  
  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKp.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fKm.FixP4(miss);
}
void K2::Topo_2(){
  //For topology Beam:Proton:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fProton,&fKm);
  fTrigger.SubtractStartTimeBeam(&fProton,&fBeam);

  fProton.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

  //Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fProton.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  fKp.FixP4(miss);
}
void K2::Topo_3(){
  //For topology Beam:Kp:Km

  //Subtract starttime from all particles
  //This has been predetermined from the tagger time
  fTrigger.SubtractStartTime(&fKp,&fKm);
  fTrigger.SubtractStartTimeBeam(&fKp,&fBeam);

  fKp.TakeCorrectedP();//Apply eloss
  fKm.TakeCorrectedP();//Apply eloss

//Reconstruct missing or combined particles
  HSLorentzVector miss= fBeam.P4() + fTarget -fKp.P4() -fKm.P4();
  TD.MissMass2=miss.M2();
  TD.MissMass=miss.M();
  
  fProton.FixP4(miss);
}

## Common Calculations : Kinematics
The final state kinematics are common to all topologies so we only need to write one function.
We can also make use of the Kinematics calculator HSKinematics (use as fKine)  for calculating standard production and decay quantities

%edit KinematicsK2.h

In [ ]:
%%file KinematicsK2.h
#include "K2.h"
void K2::Kinematics(){

  //Calculate possible resonances
  HSLorentzVector pMeson=fKp.P4()+fKm.P4();
  HSLorentzVector pHyperon=fProton.P4()+fKm.P4();
  //invariant masses
  TD.MesonMass=pMeson.M();
  TD.HyperonMass=pHyperon.M();
  
  fKine.SetGammaTarget(fBeam.P4(),fTarget);
  TD.Eg=fBeam.P4().E(); //photon bem energy

  //calculate CM production kinematics for meson
  fKine.SetMesonBaryon(pMeson,fProton.P4());
  fKine.PhotoCMDecay();
  TD.MesonCosTh=fKine.CosTheta();//prefix all variables to be saved wiht TM.
  TD.MesonPhi=fKine.Phi();
  
  //calculate CM production kinematics for hyperon
  fKine.SetMesonBaryon(fKp.P4(),pHyperon);
  fKine.PhotoCMDecay();
  TD.HyperonCosTh=fKine.CosTheta();//prefix all variables to be saved wiht TM.
  TD.HyperonPhi=fKine.Phi();

}

## Tree Data
The data you write in a tree is connected via a TreeData class which has data members which will be the tree leafs i.e. data you can perform further analysis on (e.g. histogram, event fitting) 

%edit TreeDataK2.h

In [ ]:
%%file TreeDataK2.h
#ifndef TREEDATAK2_h
#define TREEDATAK2_h 
#include "TreeData.h"

class TreeDataK2: public HS::TreeData{
 public:
  //data member for tree branches below here
  Double_t MissMass=0;
  Double_t MissMass2=0;
  
  Double_t MesonMass=0;
  Double_t HyperonMass=0;
  
  Double_t MesonCosTh=0;
  Double_t MesonPhi=0;
  Double_t HyperonCosTh=0;
  Double_t HyperonPhi=0;
  Double_t Eg=0;
  
  
  ClassDef(TreeDataK2,1);
};
#endif

Now you can try and compile all of the code together. You may need to go back and fix bugs after. This is best done by removing the %edit cells and doing a restart and run whole notebook.

In [ ]:
gROOT->ProcessLine(".x LoadK2.C+");

In [ ]:
K2 kk;